In [2]:
!pip install imbalanced-learn
!pip install kaggle

In [4]:
!cp /content/drive/MyDrive/kaggle.json /content/kaggle.json

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Check the file in its new directory
!ls /root/.kaggle/

# Check the file permission
!ls -l ~/.kaggle/kaggle.json

#Change the file permission
# chmod 600 file – owner can read and write
# chmod 700 file – owner can read, write and execute
!chmod 600 ~/.kaggle/kaggle.json

kaggle.json
-rw------- 1 root root 67 Mar 30 05:53 /root/.kaggle/kaggle.json


In [6]:
!kaggle datasets download -d mlg-ulb/creditcardfraud

 86% 57.0M/66.0M [00:00<00:00, 79.1MB/s]
100% 66.0M/66.0M [00:00<00:00, 104MB/s] 


In [7]:
import pandas as pd
import numpy as np

In [8]:
!unzip /content/creditcardfraud.zip

Archive:  /content/creditcardfraud.zip
  inflating: creditcard.csv          


In [10]:
def load_df():
  df=pd.read_csv('/content/creditcard.csv')
  return df

In [12]:
df=load_df()
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [13]:
df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [14]:
X=df.drop('Class',axis=1)
y=df['Class']

In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

## Just Simple Model


#### Logistic Regression

In [22]:
lr=LogisticRegression()
grid={'C':10.0**np.arange(-2,3),
      'penalty':['l1','l2'],}
cv=KFold(n_splits=5,random_state=None,shuffle=True)


In [ ]:
clf=GridSearchCV(lr,param_grid=grid,cv=cv,scoring='f1_macro')
clf.fit(X_train,y_train)

In [27]:
def check_result(y_test,y_pred):
  print(classification_report(y_test,y_pred))
  print(confusion_matrix(y_test,y_pred))

In [29]:
y_pred=clf .predict(X_test)
check_result(y_test,y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56869
           1       0.72      0.75      0.74        93

    accuracy                           1.00     56962
   macro avg       0.86      0.88      0.87     56962
weighted avg       1.00      1.00      1.00     56962

[[56842    27]
 [   23    70]]


In [34]:
284315/492

577.8760162601626

#### Random Forest

In [35]:
from sklearn.ensemble import RandomForestClassifier
rfr=RandomForestClassifier()
rfr.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight={0: 1, 1: 578}, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [36]:
y_pred=rfr.predict(X_test)
check_result(y_test,y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56869
           1       0.93      0.80      0.86        93

    accuracy                           1.00     56962
   macro avg       0.96      0.90      0.93     56962
weighted avg       1.00      1.00      1.00     56962

[[56863     6]
 [   19    74]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfr=RandomForestClassifier(class_weight={0:1,1:578})
rfr.fit(X_train,y_train)

In [ ]:
y_pred=rfr.predict(X_test)
check_result(y_test,y_pred)

## UnderSampling

In [37]:
from collections import Counter
from imblearn.under_sampling import NearMiss
nm=NearMiss(0.8)
X_train_nm,y_train_nm=nm.fit_sample(X_train,y_train)
print(Counter(y_train))
print(Counter(y_train_nm))

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/loca

Counter({0: 227446, 1: 399})
Counter({0: 498, 1: 399})


In [39]:
498*0.8

398.40000000000003

In [41]:
from sklearn.ensemble import RandomForestClassifier
rfr=RandomForestClassifier()
rfr.fit(X_train_nm,y_train_nm)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [42]:
y_pred=rfr.predict(X_test)
check_result(y_test,y_pred)

              precision    recall  f1-score   support

           0       1.00      0.84      0.91     56869
           1       0.01      0.92      0.02        93

    accuracy                           0.84     56962
   macro avg       0.50      0.88      0.47     56962
weighted avg       1.00      0.84      0.91     56962

[[47936  8933]
 [    7    86]]


## OverSampling

### RandomOverSampler

In [47]:
from imblearn.over_sampling import RandomOverSampler
os=RandomOverSampler(0.5)
X_train_os,y_train_os=os.fit_sample(X_train,y_train)
print(Counter(y_train))
print(Counter(y_train_os))

Counter({0: 227446, 1: 399})
Counter({0: 227446, 1: 113723})


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [48]:
rfr=RandomForestClassifier()
rfr.fit(X_train_os,y_train_os)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [49]:
y_pred=rfr.predict(X_test)
check_result(y_test,y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56869
           1       0.93      0.83      0.87        93

    accuracy                           1.00     56962
   macro avg       0.96      0.91      0.94     56962
weighted avg       1.00      1.00      1.00     56962

[[56863     6]
 [   16    77]]


## SMOTETomek

In [51]:
from imblearn.combine import SMOTETomek

In [54]:
smtk=SMOTETomek(0.75)
X_train_smtk,y_train_smtk=smtk.fit_sample(X_train,y_train)
print(Counter(y_train))
print(Counter(y_train_smtk))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Counter({0: 227446, 1: 399})
Counter({0: 226707, 1: 169845})


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [55]:
rfr=RandomForestClassifier()
rfr.fit(X_train_smtk,y_train_smtk)
y_pred=rfr.predict(X_test)
check_result(y_test,y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56869
           1       0.88      0.88      0.88        93

    accuracy                           1.00     56962
   macro avg       0.94      0.94      0.94     56962
weighted avg       1.00      1.00      1.00     56962

[[56858    11]
 [   11    82]]


## Ensemble Techniques

In [56]:
from imblearn.ensemble import EasyEnsembleClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24

In [57]:
em=EasyEnsembleClassifier(0.75)
X_train_em,y_train_em=em.fit_sample(X_train,y_train)
print(Counter(y_train))
print(Counter(y_train_em))

AttributeError: ignored

In [58]:
rfr=RandomForestClassifier()
rfr.fit(X_train_smtk,y_train_smtk)
y_pred=rfr.predict(X_test)
check_result(y_test,y_pred)

KeyboardInterrupt: ignored

## Isolation Forest